In [1]:
import os
import time
import folium
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
from turtle import color
from folium import plugins
import matplotlib.pyplot as plt
%matplotlib inline
from folium.plugins import FastMarkerCluster
pd.options.mode.chained_assignment = None  #close copy warning   


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso

In [2]:
property_df = pd.read_csv("../data/curated/data.csv")

In [3]:
print(list(property_df.columns))

['address', 'rent_weekly', 'floor', 'suburb', 'postcode', 'type', 'furnitured', 'pool', 'gym', 'num_bed', 'num_bath', 'num_car_park', 'coordinates', 'SECONDARY SCHOOL', 'PRIMARY AND SECONDARY SCHOOL', 'PRIMARY SCHOOL', 'SHOPPING CENTRE', 'PARK', 'KINDERGARTEN', 'POST OFFICE', 'SPECIAL SCHOOL', 'CHILD CARE', 'BAR', 'SPORTS COMPLEX', 'EDUCATION COMPLEX', 'CEMETERY', 'LIBRARY', 'UNIVERSITY', 'GENERAL HOSPITAL', 'POLICE STATION', 'FURTHER EDUCATION', 'PLAYGROUND', 'PLANTATION', 'WINERY', 'FARM', 'VINEYARD', 'TENNIS COURT', 'BAY', 'PIER', 'SWIMMING POOL', 'ART GALLERY', 'HARBOUR', 'COMMUNITY HEALTH CENTRE', 'MARKET', 'PRISON', 'BOTANIC GARDENS', 'SPECIALISED HOSPITAL', 'BANK', 'AGED CARE', 'SURFING SPOT', 'COAST', 'GOLF COURSE', 'HELIPORT', 'AIRPORT', 'IRON ORE PROCESSOR', 'MILL/TIMBER OPERATIONS', 'BEACH', 'TRAIN STATION', 'TRAM STATION', 'BUS', 'Education', 'Commercial', 'Environment', 'Medication', 'Secrity', 'Industrial', 'Prison', 'Transportation', 'cloest_train_station', 'cloest_SECON

In [4]:
def convert_suburb(df):
    suburb_lis = list(df['suburb'].unique())
    for i in range(1,len(suburb_lis)+1):
        df['suburb'] = df['suburb'].replace(suburb_lis[i-1],i)
    return df

property_df['type'] = property_df['type'].replace('Apartment / Unit / Flat / Penhouse', 1)
property_df['type'] = property_df['type'].replace('Studio', 2)
property_df['type'] = property_df['type'].replace('Townhouse','House')
property_df['type'] = property_df['type'].replace('House', 3)
property_df = convert_suburb(property_df)

property_df = property_df.drop(columns=['postcode'])
property_df = property_df.fillna(-1)

col_lis = ['floor', 'suburb', 'type', 'furnitured', 'pool', 'gym', 'num_bed', 'num_bath', 'num_car_park', 'SECONDARY SCHOOL', 'PRIMARY AND SECONDARY SCHOOL', 'PRIMARY SCHOOL', 'SHOPPING CENTRE', 'PARK', 'KINDERGARTEN', 'POST OFFICE', 'SPECIAL SCHOOL', 'CHILD CARE', 'BAR', 'SPORTS COMPLEX', 'EDUCATION COMPLEX', 'CEMETERY', 'LIBRARY', 'UNIVERSITY', 'GENERAL HOSPITAL', 'POLICE STATION', 'FURTHER EDUCATION', 'PLAYGROUND', 'PLANTATION', 'WINERY', 'FARM', 'VINEYARD', 'TENNIS COURT', 'BAY', 'PIER', 'SWIMMING POOL', 'ART GALLERY', 'HARBOUR', 'COMMUNITY HEALTH CENTRE', 'MARKET', 'PRISON', 'BOTANIC GARDENS', 'SPECIALISED HOSPITAL', 'BANK', 'AGED CARE', 'SURFING SPOT', 'COAST', 'GOLF COURSE', 'HELIPORT', 'AIRPORT', 'IRON ORE PROCESSOR', 'MILL/TIMBER OPERATIONS', 'BEACH', 'TRAIN STATION', 'TRAM STATION', 'BUS', 'Education', 'Commercial', 'Environment', 'Medication', 'Secrity', 'Industrial', 'Transportation', 'cloest_SECONDARY SCHOOL', 'cloest_PRIMARY AND SECONDARY SCHOOL', 'cloest_PRIMARY SCHOOL', 'cloest_SHOPPING CENTRE', 'cloest_PARK', 'cloest_KINDERGARTEN', 'cloest_POST OFFICE', 'cloest_SPECIAL SCHOOL', 'cloest_CHILD CARE', 'cloest_BAR', 'cloest_SPORTS COMPLEX', 'cloest_EDUCATION COMPLEX', 'cloest_CEMETERY', 'cloest_LIBRARY', 'cloest_UNIVERSITY', 'cloest_GENERAL HOSPITAL', 'cloest_POLICE STATION', 'cloest_FURTHER EDUCATION', 'cloest_PLAYGROUND', 'cloest_PLANTATION', 'cloest_WINERY', 'cloest_FARM', 'cloest_VINEYARD', 'cloest_TENNIS COURT', 'cloest_BAY', 'cloest_PIER', 'cloest_SWIMMING POOL', 'cloest_ART GALLERY', 'cloest_HARBOUR', 'cloest_COMMUNITY HEALTH CENTRE', 'cloest_MATERNAL AND CHILD HEALTH CENTRE', 'cloest_MARKET', 'cloest_PRISON', 'cloest_BOTANIC GARDENS', 'cloest_SPECIALISED HOSPITAL', 'cloest_BANK', 'cloest_AGED CARE', 'cloest_SURFING SPOT', 'cloest_COAST', 'cloest_GOLF COURSE', 'cloest_HELIPORT', 'cloest_AIRPORT', 'cloest_IRON ORE PROCESSOR', 'cloest_MILL/TIMBER OPERATIONS', 'cloest_BEACH', 'cloest_TRAM STATION', 'cloest_BUS', 'cloest_TRAIN_STATION_duration', 'cloest_TRAIN_STATION_distance']

In [5]:
property_df.isnull().value_counts()

address  rent_weekly  floor  suburb  type   furnitured  pool   gym    num_bed  num_bath  num_car_park  coordinates  SECONDARY SCHOOL  PRIMARY AND SECONDARY SCHOOL  PRIMARY SCHOOL  SHOPPING CENTRE  PARK   KINDERGARTEN  POST OFFICE  SPECIAL SCHOOL  CHILD CARE  BAR    SPORTS COMPLEX  EDUCATION COMPLEX  CEMETERY  LIBRARY  UNIVERSITY  GENERAL HOSPITAL  POLICE STATION  FURTHER EDUCATION  PLAYGROUND  PLANTATION  WINERY  FARM   VINEYARD  TENNIS COURT  BAY    PIER   SWIMMING POOL  ART GALLERY  HARBOUR  COMMUNITY HEALTH CENTRE  MARKET  PRISON  BOTANIC GARDENS  SPECIALISED HOSPITAL  BANK   AGED CARE  SURFING SPOT  COAST  GOLF COURSE  HELIPORT  AIRPORT  IRON ORE PROCESSOR  MILL/TIMBER OPERATIONS  BEACH  TRAIN STATION  TRAM STATION  BUS    Education  Commercial  Environment  Medication  Secrity  Industrial  Prison  Transportation  cloest_train_station  cloest_SECONDARY SCHOOL  cloest_PRIMARY AND SECONDARY SCHOOL  cloest_PRIMARY SCHOOL  cloest_SHOPPING CENTRE  cloest_PARK  cloest_KINDERGARTEN  cloes

In [6]:
property_df.corr().abs().iloc[0].to_frame().sort_values(by=['rent_weekly'],ascending=False).head(20)

,rent_weekly
rent_weekly,1.000000
TRAM STATION,0.174567
Commercial,0.156781
POST OFFICE,0.147616
SPORTS COMPLEX,0.133844
MARKET,0.123225
type,0.120412
Secrity,0.117222
POLICE STATION,0.117222
TRAIN STATION,0.113993


In [7]:
internal = property_df[['address', 'rent_weekly', 'floor', 'suburb', 'type', 'furnitured', 'pool', 'gym', 'num_bed', 'num_bath', 'num_car_park']]
external = property_df[['address', 'rent_weekly','SECONDARY SCHOOL', 'PRIMARY AND SECONDARY SCHOOL', 'PRIMARY SCHOOL', 'SHOPPING CENTRE', 'PARK', 'KINDERGARTEN', 'POST OFFICE', 'SPECIAL SCHOOL', 'CHILD CARE', 'BAR', 'SPORTS COMPLEX', 'EDUCATION COMPLEX', 'CEMETERY', 'LIBRARY', 'UNIVERSITY', 'GENERAL HOSPITAL', 'POLICE STATION', 'FURTHER EDUCATION', 'PLAYGROUND', 'PLANTATION', 'WINERY', 'FARM', 'VINEYARD', 'TENNIS COURT', 'BAY', 'PIER', 'SWIMMING POOL', 'ART GALLERY', 'HARBOUR', 'COMMUNITY HEALTH CENTRE', 'MARKET', 'PRISON', 'BOTANIC GARDENS', 'SPECIALISED HOSPITAL', 'BANK', 'AGED CARE', 'SURFING SPOT', 'COAST', 'GOLF COURSE', 'HELIPORT', 'AIRPORT', 'IRON ORE PROCESSOR', 'MILL/TIMBER OPERATIONS', 'BEACH', 'TRAIN STATION', 'TRAM STATION', 'BUS']]
dimension = property_df[['address', 'rent_weekly','Education', 'Commercial', 'Environment', 'Medication', 'Secrity', 'Industrial', 'Prison', 'Transportation']]

In [8]:
internal.corr().abs().iloc[0].to_frame().sort_values(by=['rent_weekly'],ascending=False)

,rent_weekly
rent_weekly,1.000000
type,0.120412
gym,0.109401
floor,0.101841
num_bed,0.093900
pool,0.091041
suburb,0.087228
num_car_park,0.082851
furnitured,0.036077
num_bath,0.025824


In [9]:
internal[internal['type'] == 1].corr().abs().iloc[0].to_frame().sort_values(by=['rent_weekly'],ascending=False)

,rent_weekly
rent_weekly,1.000000
suburb,0.140465
gym,0.126772
pool,0.102632
floor,0.101654
num_car_park,0.052256
num_bath,0.049523
num_bed,0.041645
furnitured,0.037907
type,NaN


In [10]:
internal[internal['type'] == 3].corr().abs().iloc[0].to_frame().sort_values(by=['rent_weekly'],ascending=False)

,rent_weekly
rent_weekly,1.000000
num_car_park,0.032713
pool,0.023810
suburb,0.023093
num_bed,0.015205
floor,0.008797
gym,0.008449
furnitured,0.003345
num_bath,0.002202
type,NaN


In [11]:
external.corr().abs().iloc[0].to_frame().sort_values(by=['rent_weekly'],ascending=False).head(15)

,rent_weekly
rent_weekly,1.000000
TRAM STATION,0.174567
POST OFFICE,0.147616
SPORTS COMPLEX,0.133844
MARKET,0.123225
POLICE STATION,0.117222
TRAIN STATION,0.113993
BOTANIC GARDENS,0.096230
SECONDARY SCHOOL,0.091323
GENERAL HOSPITAL,0.086397


In [12]:
dimension.corr().abs().iloc[0].to_frame().sort_values(by=['rent_weekly'],ascending=False).head(15)

,rent_weekly
rent_weekly,1.000000
Commercial,0.156781
Secrity,0.117222
Medication,0.083742
Transportation,0.068083
Prison,0.066797
Environment,0.050913
Education,0.049204
Industrial,0.032781


In [13]:
X = property_df[col_lis].copy()
y = property_df['rent_weekly']
X_norm = MinMaxScaler().fit_transform(X)

In [14]:
mi_selector = SelectKBest(mutual_info_classif, k=len(col_lis))
mi_selector.fit(X, y)
mi_scores = mi_selector.scores_
mi_support = mi_selector.get_support()
mi_feature = X.loc[:,mi_support].columns.tolist()
idx = [x for x in range(len(mi_support)) if mi_support[x]]
score = {}
for i in idx:
    score[col_lis[i]] = mi_scores[i]
pd.DataFrame.from_dict({'features':score.keys(), 'mi_value':score.values()}).sort_values(by=['mi_value'], ascending=False)

,features,mi_value
1,suburb,0.186841
98,cloest_BANK,0.102605
101,cloest_COAST,0.095675
100,cloest_SURFING SPOT,0.088715
99,cloest_AGED CARE,0.085103
...,...,...
47,GOLF COURSE,0.000000
32,TENNIS COURT,0.000000
29,WINERY,0.000000
21,CEMETERY,0.000000


In [15]:
chi_selector = SelectKBest(chi2, k=len(col_lis))
chi_selector.fit(X_norm, y)
chi_scores = chi_selector.scores_
chi_pvalues = chi_selector.pvalues_
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
idx = [x for x in range(len(chi_support)) if chi_support[x]]
score = {}
for i in idx:
    score[col_lis[i]] = chi_scores[i]
print(score)

pvalue = {}
for i in idx:
    pvalue[col_lis[i]] = chi_pvalues[i]
print(pvalue)

pd.DataFrame.from_dict({'features':score.keys(), 'chi2_value':score.values()}).sort_values(by=['chi2_value'], ascending=False)

{'floor': 71.90296881216268, 'suburb': 75.91850063398181, 'type': 186.53432955785905, 'furnitured': 337.875201938322, 'pool': 411.6836493962104, 'gym': 470.62036853402293, 'num_bed': 36.74334449816516, 'num_bath': 7.841107689746503, 'num_car_park': 30.645099441614473, 'SECONDARY SCHOOL': 126.10617140483872, 'PRIMARY AND SECONDARY SCHOOL': 49.389452717461126, 'PRIMARY SCHOOL': 48.155231643204644, 'SHOPPING CENTRE': 145.8109968281035, 'PARK': 41.93655308475271, 'KINDERGARTEN': 47.961666895040835, 'POST OFFICE': 278.503594425298, 'SPECIAL SCHOOL': 156.89779706535472, 'CHILD CARE': 83.2723676563039, 'BAR': 323.1257806418001, 'SPORTS COMPLEX': 370.96124161967447, 'EDUCATION COMPLEX': 176.37151194095384, 'CEMETERY': 174.05487699155339, 'LIBRARY': 213.67348684484728, 'UNIVERSITY': 245.15242936719955, 'GENERAL HOSPITAL': 107.1636489357276, 'POLICE STATION': 194.3923961728371, 'FURTHER EDUCATION': 128.3470806449179, 'PLAYGROUND': 151.46856323979625, 'PLANTATION': 14.643127863615863, 'WINERY': 5

,features,chi2_value
5,gym,470.620369
39,MARKET,440.673506
43,BANK,423.939057
4,pool,411.683649
41,BOTANIC GARDENS,377.923148
...,...,...
44,AGED CARE,NaN
45,SURFING SPOT,NaN
46,COAST,NaN
48,HELIPORT,NaN


In [16]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

cor_support, cor_feature = cor_selector(X, y,len(col_lis))
print(str(len(cor_feature)), 'selected features')

112 selected features


/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/lucinda/opt/anaconda3/envs/ads_env/lib/python3.8/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(max_iter=10000), n_features_to_select=len(col_lis), step=10, verbose=5)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()


In [18]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(solver='liblinear',penalty="l1",max_iter=10000), max_features=len(col_lis))
embeded_lr_selector.fit(X_norm, y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()


In [19]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=len(col_lis))
embeded_rf_selector.fit(X, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()

In [ ]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=len(col_lis))
embeded_lgb_selector.fit(X, y)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()

In [ ]:

# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)

In [ ]:
X = property_df[property_df['type'] == 1].copy()
X = property_df[col_lis]
y = property_df['rent_weekly']
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=15)
chi_selector.fit(X_norm, y)
chi_scores = chi_selector.scores_
chi_pvalues = chi_selector.pvalues_
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
idx = [x for x in range(len(chi_support)) if chi_support[x]]
score = {}
for i in idx:
    score[col_lis[i]] = chi_scores[i]
print(score)

pvalue = {}
for i in idx:
    pvalue[col_lis[i]] = chi_pvalues[i]
print(pvalue)

pd.DataFrame.from_dict({'features':score.keys(), 'chi2_value':score.values()}).sort_values(by=['chi2_value'], ascending=False)

In [ ]:
X = property_df[property_df['type'] == 3].copy()
X = property_df[col_lis]
y = property_df['rent_weekly']
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=15)
chi_selector.fit(X_norm, y)
chi_scores = chi_selector.scores_
chi_pvalues = chi_selector.pvalues_
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
idx = [x for x in range(len(chi_support)) if chi_support[x]]
score = {}
for i in idx:
    score[col_lis[i]] = chi_scores[i]
print(score)

pvalue = {}
for i in idx:
    pvalue[col_lis[i]] = chi_pvalues[i]
print(pvalue)

pd.DataFrame.from_dict({'features':score.keys(), 'chi2_value':score.values()}).sort_values(by=['chi2_value'], ascending=False)